In [ ]:
%reset -f

import json
import pandas as pd
import os
from datetime import datetime, timedelta

year = 2015 # Choose year

directory_path = fr'C:\Users\peter\Downloads\{year}' # Choose path

all_data_df = pd.DataFrame()

start_date = datetime(year, 1, 1)
end_date = datetime(year, 12, 31)

print(directory_path)
current_date = start_date

while current_date <= end_date:
    file_name = current_date.strftime('%Y-%m-%d') + '.txt'
    file_path = os.path.join(directory_path, file_name)
    
    if os.path.exists(file_path):
        data_list = []

        # Open the file and read line by line
        with open(file_path, 'r') as file:
            for line in file:
                # Parse each line as a JSON object
                data = json.loads(line)
                # Extract the relevant fields from 'properties'
                properties = data['properties']
                data_list.append({
                    'parameterId': properties['parameterId'],
                    'stationId': properties['stationId'],
                    'observed': properties['observed'],
                    'value': properties['value']
                })
        
        day_df = pd.DataFrame(data_list)
        
        # Filter for wind_speed and sun_last10min_glob in the loop to avoid accumulating unnecessary data
        day_df = day_df[(day_df['parameterId'] == 'wind_speed') | (day_df['parameterId'] == 'sun_last10min_glob')]
        
        # Convert 'observed' to datetime and filter only whole hour data
        day_df['observed'] = pd.to_datetime(day_df['observed'])
        day_df = day_df[day_df['observed'].dt.minute == 0]
        
        # Append 
        all_data_df = pd.concat([all_data_df, day_df], ignore_index=True)
    
    current_date += timedelta(days=1)

    print(current_date)

# There are some duplicate values that I take the mean of
wind_df = all_data_df[all_data_df['parameterId'] == 'wind_speed']
wind_df = wind_df.groupby(['observed', 'stationId']).agg({'value': 'mean'}).reset_index()

sun_df = all_data_df[all_data_df['parameterId'] == 'sun_last10min_glob']
sun_df = sun_df.groupby(['observed', 'stationId']).agg({'value': 'mean'}).reset_index()

# Pivot the data so that each stationId has separate columns for both wind and sun data
wind_pivot = wind_df.pivot(index='observed', columns='stationId', values='value').add_prefix('wind_')
sun_pivot = sun_df.pivot(index='observed', columns='stationId', values='value').add_prefix('sun_')

result_df = wind_pivot.merge(sun_pivot, on='observed', how='outer')

result_df.reset_index(inplace=True)

result_df.to_csv(fr'Data\{year}.csv', index=False)
